# Preprocessing for prediction

In [128]:
%load_ext autoreload
%autoreload 2

!python --version

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Python 3.7.6


In [129]:
import sys

sys.path.append('..')

In [130]:
# show full pandas output when using df.describe()

import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from src.constants import SPECIFIC_POSITIONS, NUMERIC_ATTRIBUTES
from sklearn import preprocessing


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [131]:
from sklearn.pipeline import Pipeline, make_pipeline

from src.constants import SKILL_ATTRIBUTES
from src.helpers import get_nan_columns

from src.preprocessing.transformers import FeatureSelector, FilterPlayersWithNanValues

from src.preprocessing.pandas_feature_union import PandasFeatureUnion
from src.preprocessing.pandas_one_hot_encoder import PandasOneHotEncoder

In [132]:
data = pd.read_csv('../data/fifa_processed.csv')

In [133]:
data.head()

,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause,LS Value,LS Grow,ST Value,ST Grow,RS Value,RS Grow,LW Value,LW Grow,LF Value,LF Grow,CF Value,CF Grow,RW Value,RW Grow,LAM Value,LAM Grow,RAM Value,RAM Grow,LM Value,LM Grow,LCM Value,LCM Grow,CM Value,CM Grow,RCM Value,RCM Grow,RM Value,RM Grow,LWB Value,LWB Grow,LDM Value,LDM Grow,RDM Value,RDM Grow,LB Value,LB Grow,LCB Value,LCB Grow,RCB Value,RCB Grow,RB Value,RB Grow,Work Rate Attack,Work Rate Defense,Position (4),Position (13),Contract Length
0,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,https://cdn.sofifa.org/teams/2/light/241.png,110500000.0,565000.0,2202,Left,5.0,4.0,4.0,Medium/ Medium,Messi,True,RF,10.0,1546300800,NaN,1609459200,176.76,159.0,88+2,88+2,88+2,92+2,93+2,93+2,93+2,92+2,93+2,93+2,93+2,91+2,84+2,84+2,84+2,91+2,64+2,61+2,61+2,61+2,64+2,59+2,47+2,47+2,47+2,59+2,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,226500000.0,88.0,2.0,88.0,2.0,88.0,2.0,92.0,2.0,93.0,2.0,93.0,2.0,92.0,2.0,93.0,2.0,93.0,2.0,91.0,2.0,84.0,2.0,84.0,2.0,84.0,2.0,91.0,2.0,64.0,2.0,61.0,2.0,61.0,2.0,59.0,2.0,47.0,2.0,47.0,2.0,59.0,2.0,Medium,Medium,attack,right attack,63158400.0
1,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,https://cdn.sofifa.org/teams/2/light/45.png,77000000.0,405000.0,2228,Right,5.0,4.0,5.0,High/ Low,C. Ronaldo,True,ST,7.0,1546300800,NaN,1640995200,189.84,183.0,91+3,91+3,91+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,127100000.0,91.0,3.0,91.0,3.0,91.0,3.0,89.0,3.0,90.0,3.0,90.0,3.0,89.0,3.0,88.0,3.0,88.0,3.0,88.0,3.0,81.0,3.0,81.0,3.0,81.0,3.0,88.0,3.0,65.0,3.0,61.0,3.0,61.0,3.0,61.0,3.0,53.0,3.0,53.0,3.0,61.0,3.0,High,Low,attack,central attack,94694400.0
2,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,93,Paris Saint-Germain,https://cdn.sofifa.org/teams/2/light/73.png,118500000.0,290000.0,2143,Right,5.0,5.0,5.0,High/ Medium,Neymar,True,LW,10.0,1546300800,NaN,1640995200,183.72,150.0,84+3,84+3,84+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,88+3,81+3,81+3,81+3,88+3,65+3,60+3,60+3,60+3,65+3,60+3,47+3,47+3,47+3,60+3,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,228100000.0,84.0,3.0,84.0,3.0,84.0,3.0,89.0,3.0,89.0,3.0,89.0,3.0,89.0,3.0,89.0,3.0,89.0,3.0,88.0,3.0,81.0,3.0,81.0,3.0,81.0,3.0,88.0,3.0,65.0,3.0,60.0,3.0,60.0,3.0,60.0,3.0,47.0,3.0,47.0,3.0,60.0,3.0,High,Medium,attack,left attack,94694400.0
3,De Gea,27,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91,93,Manchester United,https://cdn.sofifa.org/teams/2/light/11.png,72000000.0,260000.0,1471,Right,4.0,3.0,1.0,Medium/ Medium,Lean,True,GK,1.0,1546300800,NaN,1577836800,196.80,168.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [134]:
# classification task 1 (selected attributes)

cls_categoric_features = ['Preferred Foot', 'Work Rate Attack', 'Work Rate Defense']
cls_numeric_features = np.concatenate([SKILL_ATTRIBUTES, ['Age', 'Overall', 'Potential', 'Weak Foot', 'Skill Moves', 'Height', 'Weight']], axis=0)
cls_features = np.concatenate([cls_categoric_features, cls_numeric_features], axis=0)

In [135]:
# regression task 2 (selected attributes)

reg_categoric_features = np.concatenate([['Position (4)', 'Position (13)'], cls_categoric_features], axis=0)
reg_numeric_features = np.concatenate([['Contract Length', 'International Reputation'], cls_numeric_features], axis=0)
reg_features = np.concatenate([reg_categoric_features, reg_numeric_features], axis=0)

In [136]:
class FeatureNormalizer(BaseEstimator, TransformerMixin):
    def __init__(self, cols):
        self.cols = cols
    
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        assert isinstance(X, pd.DataFrame)
        
        for col in self.cols:
            float_col = X[[col]].values.astype(float)
            min_max_scaler = preprocessing.MinMaxScaler()
            col_scaled = min_max_scaler.fit_transform(float_col)
            X[col] = col_scaled
        return X

In [137]:
specific_positions_value = list(map(lambda x: x + ' Value', SPECIFIC_POSITIONS))
specific_positions_grow = list(map(lambda x: x + ' Grow', SPECIFIC_POSITIONS))

## Pipeline for classification

In [138]:
data[data['Position'].notna()]

,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause,LS Value,LS Grow,ST Value,ST Grow,RS Value,RS Grow,LW Value,LW Grow,LF Value,LF Grow,CF Value,CF Grow,RW Value,RW Grow,LAM Value,LAM Grow,RAM Value,RAM Grow,LM Value,LM Grow,LCM Value,LCM Grow,CM Value,CM Grow,RCM Value,RCM Grow,RM Value,RM Grow,LWB Value,LWB Grow,LDM Value,LDM Grow,RDM Value,RDM Grow,LB Value,LB Grow,LCB Value,LCB Grow,RCB Value,RCB Grow,RB Value,RB Grow,Work Rate Attack,Work Rate Defense,Position (4),Position (13),Contract Length
0,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,https://cdn.sofifa.org/teams/2/light/241.png,110500000.0,565000.0,2202,Left,5.0,4.0,4.0,Medium/ Medium,Messi,True,RF,10.0,1546300800,NaN,1609459200,176.76,159.0,88+2,88+2,88+2,92+2,93+2,93+2,93+2,92+2,93+2,93+2,93+2,91+2,84+2,84+2,84+2,91+2,64+2,61+2,61+2,61+2,64+2,59+2,47+2,47+2,47+2,59+2,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,226500000.0,88.0,2.0,88.0,2.0,88.0,2.0,92.0,2.0,93.0,2.0,93.0,2.0,92.0,2.0,93.0,2.0,93.0,2.0,91.0,2.0,84.0,2.0,84.0,2.0,84.0,2.0,91.0,2.0,64.0,2.0,61.0,2.0,61.0,2.0,59.0,2.0,47.0,2.0,47.0,2.0,59.0,2.0,Medium,Medium,attack,right attack,63158400.0
1,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,https://cdn.sofifa.org/teams/2/light/45.png,77000000.0,405000.0,2228,Right,5.0,4.0,5.0,High/ Low,C. Ronaldo,True,ST,7.0,1546300800,NaN,1640995200,189.84,183.0,91+3,91+3,91+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,127100000.0,91.0,3.0,91.0,3.0,91.0,3.0,89.0,3.0,90.0,3.0,90.0,3.0,89.0,3.0,88.0,3.0,88.0,3.0,88.0,3.0,81.0,3.0,81.0,3.0,81.0,3.0,88.0,3.0,65.0,3.0,61.0,3.0,61.0,3.0,61.0,3.0,53.0,3.0,53.0,3.0,61.0,3.0,High,Low,attack,central attack,94694400.0
2,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,93,Paris Saint-Germain,https://cdn.sofifa.org/teams/2/light/73.png,118500000.0,290000.0,2143,Right,5.0,5.0,5.0,High/ Medium,Neymar,True,LW,10.0,1546300800,NaN,1640995200,183.72,150.0,84+3,84+3,84+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,88+3,81+3,81+3,81+3,88+3,65+3,60+3,60+3,60+3,65+3,60+3,47+3,47+3,47+3,60+3,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,228100000.0,84.0,3.0,84.0,3.0,84.0,3.0,89.0,3.0,89.0,3.0,89.0,3.0,89.0,3.0,89.0,3.0,89.0,3.0,88.0,3.0,81.0,3.0,81.0,3.0,81.0,3.0,88.0,3.0,65.0,3.0,60.0,3.0,60.0,3.0,60.0,3.0,47.0,3.0,47.0,3.0,60.0,3.0,High,Medium,attack,left attack,94694400.0
3,De Gea,27,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91,93,Manchester United,https://cdn.sofifa.org/teams/2/light/11.png,72000000.0,260000.0,1471,Right,4.0,3.0,1.0,Medium/ Medium,Lean,True,GK,1.0,1546300800,NaN,1577836800,196.80,168.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [139]:
data[data['Position'].notna()]

,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause,LS Value,LS Grow,ST Value,ST Grow,RS Value,RS Grow,LW Value,LW Grow,LF Value,LF Grow,CF Value,CF Grow,RW Value,RW Grow,LAM Value,LAM Grow,RAM Value,RAM Grow,LM Value,LM Grow,LCM Value,LCM Grow,CM Value,CM Grow,RCM Value,RCM Grow,RM Value,RM Grow,LWB Value,LWB Grow,LDM Value,LDM Grow,RDM Value,RDM Grow,LB Value,LB Grow,LCB Value,LCB Grow,RCB Value,RCB Grow,RB Value,RB Grow,Work Rate Attack,Work Rate Defense,Position (4),Position (13),Contract Length
0,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,https://cdn.sofifa.org/teams/2/light/241.png,110500000.0,565000.0,2202,Left,5.0,4.0,4.0,Medium/ Medium,Messi,True,RF,10.0,1546300800,NaN,1609459200,176.76,159.0,88+2,88+2,88+2,92+2,93+2,93+2,93+2,92+2,93+2,93+2,93+2,91+2,84+2,84+2,84+2,91+2,64+2,61+2,61+2,61+2,64+2,59+2,47+2,47+2,47+2,59+2,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,226500000.0,88.0,2.0,88.0,2.0,88.0,2.0,92.0,2.0,93.0,2.0,93.0,2.0,92.0,2.0,93.0,2.0,93.0,2.0,91.0,2.0,84.0,2.0,84.0,2.0,84.0,2.0,91.0,2.0,64.0,2.0,61.0,2.0,61.0,2.0,59.0,2.0,47.0,2.0,47.0,2.0,59.0,2.0,Medium,Medium,attack,right attack,63158400.0
1,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,https://cdn.sofifa.org/teams/2/light/45.png,77000000.0,405000.0,2228,Right,5.0,4.0,5.0,High/ Low,C. Ronaldo,True,ST,7.0,1546300800,NaN,1640995200,189.84,183.0,91+3,91+3,91+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,127100000.0,91.0,3.0,91.0,3.0,91.0,3.0,89.0,3.0,90.0,3.0,90.0,3.0,89.0,3.0,88.0,3.0,88.0,3.0,88.0,3.0,81.0,3.0,81.0,3.0,81.0,3.0,88.0,3.0,65.0,3.0,61.0,3.0,61.0,3.0,61.0,3.0,53.0,3.0,53.0,3.0,61.0,3.0,High,Low,attack,central attack,94694400.0
2,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,93,Paris Saint-Germain,https://cdn.sofifa.org/teams/2/light/73.png,118500000.0,290000.0,2143,Right,5.0,5.0,5.0,High/ Medium,Neymar,True,LW,10.0,1546300800,NaN,1640995200,183.72,150.0,84+3,84+3,84+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,88+3,81+3,81+3,81+3,88+3,65+3,60+3,60+3,60+3,65+3,60+3,47+3,47+3,47+3,60+3,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,228100000.0,84.0,3.0,84.0,3.0,84.0,3.0,89.0,3.0,89.0,3.0,89.0,3.0,89.0,3.0,89.0,3.0,89.0,3.0,88.0,3.0,81.0,3.0,81.0,3.0,81.0,3.0,88.0,3.0,65.0,3.0,60.0,3.0,60.0,3.0,60.0,3.0,47.0,3.0,47.0,3.0,60.0,3.0,High,Medium,attack,left attack,94694400.0
3,De Gea,27,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91,93,Manchester United,https://cdn.sofifa.org/teams/2/light/11.png,72000000.0,260000.0,1471,Right,4.0,3.0,1.0,Medium/ Medium,Lean,True,GK,1.0,1546300800,NaN,1577836800,196.80,168.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [140]:
cls_pipeline = Pipeline([
    ('filter_players_with_nan_values', FilterPlayersWithNanValues(['Position'])),
    ('transform', PandasFeatureUnion([
        ('categoric_features', make_pipeline(
            FeatureSelector(cls_categoric_features),
            PandasOneHotEncoder(),
        )),
        ('numeric_features', make_pipeline(
            FeatureSelector(cls_numeric_features),
        )),
        ('class', make_pipeline(
            FeatureSelector(['Position (4)', 'Position (13)']),
        ))
    ])),
    ('normalize_numeric_cols', FeatureNormalizer(cls_numeric_features))
])

In [141]:
cls_data = cls_pipeline.fit_transform(data)

In [142]:
cls_data.head()

,Preferred Foot_Left,Preferred Foot_Right,Preferred Foot_nan,Work Rate Attack_High,Work Rate Attack_Low,Work Rate Attack_Medium,Work Rate Attack_nan,Work Rate Defense_High,Work Rate Defense_Low,Work Rate Defense_Medium,Work Rate Defense_nan,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Age,Overall,Potential,Weak Foot,Skill Moves,Height,Weight,Position (4),Position (13)
0,1,0,0,0,0,1,0,0,0,1,0,0.897727,1.000000,0.733333,0.965116,0.953488,1.000000,0.988636,1.000000,0.928571,1.000000,0.929412,0.880952,0.939024,0.986667,0.9875,0.892473,0.6625,0.714286,0.5250,1.000000,0.440476,0.213483,0.989247,1.000000,0.804598,1.000000,0.329670,0.285714,0.261364,0.056180,0.109890,0.155556,0.146067,0.075269,0.517241,1.000000,0.978723,0.75,0.75,0.358025,0.368421,attack,right attack
1,0,1,0,1,0,0,0,0,1,0,0,0.897727,0.989247,0.944444,0.860465,0.965116,0.903226,0.852273,0.802198,0.809524,0.978022,0.905882,0.940476,0.890244,1.000000,0.6750,1.000000,1.0000,0.904762,0.7750,0.989011,0.619048,0.292135,1.000000,0.857143,0.919540,0.989247,0.274725,0.318681,0.227273,0.067416,0.109890,0.155556,0.146067,0.107527,0.586207,1.000000,0.978723,0.75,1.00,0.582305,0.548872,attack,central attack
2,0,1,0,1,0,0,0,0,0,1,0,0.840909,0.913978,0.644444,0.895349,0.930233,0.989247,0.931818,0.923077,0.821429,0.989011,0.964706,0.928571,1.000000,0.973333,0.8500,0.838710,0.5750,0.821429,0.4000,0.868132,0.535714,0.370787,0.935484,0.916667,0.873563,0.978495,0.263736,0.241758,0.340909,0.089888,0.087912,0.155556,0.157303,0.107527,0.344828,0.958333,0.957447,1.00,1.00,0.477366,0.300752,attack,left attack
3,0,1,0,0,0,1,0,0,0,1,0,0.136364,0.118280,0.188889,0.500000,0.104651,0.150538,0.170455,0.175824,0.500000,0.406593,0.529412,0.547619,0.560976,0.920000,0.3375,0.311828,0.6500,0.369048,0.5875,0.098901,0.321429,0.303371,0.107527,0.690476,0.402299,0.698925,0.131868,0.208791,0.113636,1.000000,0.923077,0.955556,0.977528,1.000000,0.379310,0.937500,0.957447,0.50,0.00,0.701646,0.436090,goalkeeper,goalkeeper
4,0,1,0,1,0,0,0,1,0,0,0,1.000000,0.860215,0.566667,0.988372,0.906977,0.881720,0.897727,0.879121,0.976190,0.945055,0.776471,0.761905,0.792683,0.933333,0.7625,0.956989,0.6000,0.928571,0.7250,0.967033,0.773810,0.651685,0.913978,1.000000,0.850575,0.913978,0.714286,0.615385,0.545455,0.157303,0.131868,0.044444,0.101124,0.129032,0.379310,0.937500,0.936170,1.00,0.75,0.596708,0.330827,middle,midfielder


In [143]:
cls_data.to_csv('../data/fifa_processed_for_cls.csv', index=False)

## Pipeline for regression

In [144]:
reg_pipeline = Pipeline([
    ('filter_players_with_nan_values', FilterPlayersWithNanValues(['Value'])),
    ('transform', PandasFeatureUnion([
        ('categoric_features', make_pipeline(
            FeatureSelector(reg_categoric_features),
            PandasOneHotEncoder(),
        )),
        ('numeric_features', make_pipeline(
            FeatureSelector(reg_numeric_features),
        )),
        ('value', make_pipeline(
            FeatureSelector(['Value']),
        ))
    ])),
    ('normalize_numeric_cols', FeatureNormalizer(reg_numeric_features))
])

In [145]:
reg_data = reg_pipeline.fit_transform(data)

In [146]:
reg_data.head()

,Position (4)_attack,Position (4)_defense,Position (4)_goalkeeper,Position (4)_middle,Position (4)_nan,Position (13)_attacking midfielder,Position (13)_central attack,Position (13)_central defense,Position (13)_deffensive midfielder,Position (13)_goalkeeper,Position (13)_left attack,Position (13)_left defense,Position (13)_left midfielder,Position (13)_middle,Position (13)_midfielder,Position (13)_right attack,Position (13)_right defense,Position (13)_right midfielder,Position (13)_nan,Preferred Foot_Left,Preferred Foot_Right,Preferred Foot_nan,Work Rate Attack_High,Work Rate Attack_Low,Work Rate Attack_Medium,Work Rate Attack_nan,Work Rate Defense_High,Work Rate Defense_Low,Work Rate Defense_Medium,Work Rate Defense_nan,Contract Length,International Reputation,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Age,Overall,Potential,Weak Foot,Skill Moves,Height,Weight,Value
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0.375086,1.00,0.897727,1.000000,0.733333,0.965116,0.953488,1.000000,0.988636,1.000000,0.928571,1.000000,0.929412,0.880952,0.939024,0.986667,0.9875,0.892473,0.6625,0.714286,0.5250,1.000000,0.440476,0.213483,0.989247,1.000000,0.804598,1.000000,0.329670,0.285714,0.261364,0.056180,0.109890,0.155556,0.146067,0.075269,0.517241,1.000000,0.978723,0.75,0.75,0.358025,0.368421,110500000.0
1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0.500000,1.00,0.897727,0.989247,0.944444,0.860465,0.965116,0.903226,0.852273,0.802198,0.809524,0.978022,0.905882,0.940476,0.890244,1.000000,0.6750,1.000000,1.0000,0.904762,0.7750,0.989011,0.619048,0.292135,1.000000,0.857143,0.919540,0.989247,0.274725,0.318681,0.227273,0.067416,0.109890,0.155556,0.146067,0.107527,0.586207,1.000000,0.978723,0.75,1.00,0.582305,0.548872,77000000.0
2,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0.500000,1.00,0.840909,0.913978,0.644444,0.895349,0.930233,0.989247,0.931818,0.923077,0.821429,0.989011,0.964706,0.928571,1.000000,0.973333,0.8500,0.838710,0.5750,0.821429,0.4000,0.868132,0.535714,0.370787,0.935484,0.916667,0.873563,0.978495,0.263736,0.241758,0.340909,0.089888,0.087912,0.155556,0.157303,0.107527,0.344828,0.958333,0.957447,1.00,1.00,0.477366,0.300752,118500000.0
3,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0.249829,0.75,0.136364,0.118280,0.188889,0.500000,0.104651,0.150538,0.170455,0.175824,0.500000,0.406593,0.529412,0.547619,0.560976,0.920000,0.3375,0.311828,0.6500,0.369048,0.5875,0.098901,0.321429,0.303371,0.107527,0.690476,0.402299,0.698925,0.131868,0.208791,0.113636,1.000000,0.923077,0.955556,0.977528,1.000000,0.379310,0.937500,0.957447,0.50,0.00,0.701646,0.436090,72000000.0
4,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0.624914,0.75,1.000000,0.860215,0.566667,0.988372,0.906977,0.881720,0.897727,0.879121,0.976190,0.945055,0.776471,0.761905,0.792683,0.933333,0.7625,0.956989,0.6000,0.928571,0.7250,0.967033,0.773810,0.651685,0.913978,1.000000,0.850575,0.913978,0.714286,0.615385,0.545455,0.157303,0.131868,0.044444,0.101124,0.129032,0.379310,0.937500,0.936170,1.00,0.75,0.596708,0.330827,102000000.0


In [147]:
reg_data.to_csv('../data/fifa_processed_for_reg.csv', index=False)